In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.vision import *
# from fastai.callbacks import *

import torch
from torch import nn
from dsin.ae.data_manager.data_loader import (
    SideinformationImageImageList, ImageSiTuple)
from dsin.ae import config
from dsin.ae.base_ae import BaseAutoEncoder
from dsin.ae.si_net import SiNetChannelIn
from dsin.ae.loss_man import LossManager
from dsin.ae.distortions import Distortions, DistTypes
from dsin.ae.kitti_normalizer import ChangeImageStatsToKitti, ChangeState
from dsin.ae import config

from dsin.train_utils.visibilty_utils.logger_cb_utils import setup_file_logger
from dsin.train_utils.visibilty_utils.pbar_cb_utils import (AverageMetric,
                                            ParameterMetricCallback,
                                            ProgressBarAttibuteVisualizer,
                                            ParameterRunningAverageMetricCallback,
                                            BitEntropy
                                           )
from dsin.train_utils.save_utils import FileNameGenerator

In [3]:
bpp = 0.3
H_target = 2* bpp
# config.autoencoder_loss_distortion_to_minimize=DistTypes.MS_SSMIM
config.K_MS_SSIM=500
config.beta

500

In [4]:
logger = setup_file_logger()

In [5]:
config.use_si_flag = SiNetChannelIn.WithSideInformation

In [6]:
learn = None
gc.collect()

99

In [7]:
baseline_autoencoder = BaseAutoEncoder()
path = "../src/dsin/data"
pct= 1 # 0.0005 #0.25

valid_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_baseline_val.txt"],pct=pct/10)
train_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_baseline_train.txt"],pct=pct)

image_lists = ItemLists(
    path=path, train=train_image_list, valid=valid_image_list)


tfms =  get_transforms(do_flip=True,
                       flip_vert=False,
                       max_rotate=False,
                       max_zoom=0,
                       max_lighting=False,
                       max_warp=False)
batchsize = 32

data = (image_lists
        .label_from_func(lambda x: x)
        .transform(tfms, size=(192, 192), resize_method=ResizeMethod.CROP, tfm_y=True)
        .databunch(bs=batchsize))
learn = None
gc.collect()
learn = Learner(data=data,
                     model=baseline_autoencoder,
                     opt_func=torch.optim.Adam,
                     loss_func=LossManager(baseline_autoencoder,SiNetChannelIn.NoSideInformation,target_bit_cost = H_target),
                     metrics=[AverageMetric(Distortions._calc_dist,"MS_SSIM")])

In [8]:
model_fname =FileNameGenerator.create_fname(f'try2MSSIM-l2reg-baseline-bpp0p{bpp*100:.0f}')
lrs = [1e-4] *3 + [1e-5] * 4 + [1e-6] *2 
model_fname

'2020-09-04try2MSSIM-l2reg-baseline-bpp0p30'

In [ ]:
config.si_loss_weight_alpha = 0
learn.model.use_side_infomation = SiNetChannelIn.NoSideInformation
learn.loss_func.use_side_infomation = SiNetChannelIn.NoSideInformation
    
offset = 0
for i,lr in zip(range(offset,len(lrs)),lrs[offset:]):
    if i != 0 :
        learn.load(f'{model_fname}-{i - 1}',with_opt=True)
    learn.fit(1, lr=lr,wd=0,callbacks=[ParameterRunningAverageMetricCallback(learn.loss_func,use_si=False),BitEntropy(loss_man=learn.loss_func,logger=logger,use_si=False)])
    
    learn.save(f'{model_fname}-{i}')
    if i % 3 or i == len(lrs) - 1:
        !aws s3 cp ~/tDSIN/src/dsin/data/models/{model_fname}-{i}.pth  s3://dsin-us/models/
        !aws s3 cp ~/tDSIN/tutorials/out.log s3://dsin-us/models/{model_fname}.log


epoch,train_loss,valid_loss,MS_SSIM,time
0,632.952759,608.916016,0.126564,08:28


iter 0: bpp = 0.101 imp-mean-var(0.51 0.25) total loss1652.5  l2reg_loss=18.4autoencoder_loss_value=927.8si_loss=0.0feat_loss_value=0.0
iter 500: bpp = 0.909 imp-mean-var(0.42 0.24) total loss765.4  l2reg_loss=18.3autoencoder_loss_value=142.6si_loss=0.0feat_loss_value=0.0
iter 1000: bpp = 0.767 imp-mean-var(0.30 0.21) total loss616.2  l2reg_loss=18.0autoencoder_loss_value=128.8si_loss=0.0feat_loss_value=0.0


epoch,train_loss,valid_loss,MS_SSIM,time
0,222.333099,216.950653,0.127046,08:31


iter 0: bpp = 0.078 imp-mean-var(0.33 0.22) total loss634.7  l2reg_loss=18.0autoencoder_loss_value=136.3si_loss=0.0feat_loss_value=0.0
iter 500: bpp = 0.468 imp-mean-var(0.10 0.08) total loss318.7  l2reg_loss=17.8autoencoder_loss_value=144.2si_loss=0.0feat_loss_value=0.0
iter 1000: bpp = 0.369 imp-mean-var(0.09 0.08) total loss214.5  l2reg_loss=17.5autoencoder_loss_value=129.7si_loss=0.0feat_loss_value=0.0
upload: ../src/dsin/data/models/2020-09-04try2MSSIM-l2reg-baseline-bpp0p30-1.pth to s3://dsin-us/models/2020-09-04try2MSSIM-l2reg-baseline-bpp0p30-1.pth
upload: ./out.log to s3://dsin-us/models/2020-09-04try2MSSIM-l2reg-baseline-bpp0p30.log


epoch,train_loss,valid_loss,MS_SSIM,time


iter 0: bpp = 0.037 imp-mean-var(0.09 0.08) total loss223.7  l2reg_loss=17.5autoencoder_loss_value=133.7si_loss=0.0feat_loss_value=0.0
iter 500: bpp = 0.318 imp-mean-var(0.09 0.08) total loss177.2  l2reg_loss=17.3autoencoder_loss_value=124.2si_loss=0.0feat_loss_value=0.0


In [ ]:
learn.save(f'{model_fname}-{i}')

!aws s3 cp ~/tDSIN/src/dsin/data/models/{model_fname}-{i}.pth  s3://dsin-us/models/
!aws s3 cp ~/tDSIN/tutorials/out.log s3://dsin-us/models/{model_fname}.log


In [ ]:
mt = learn.model.my_tuple[-3][0,...].squeeze_().detach()
mx, mn =torch.max(mt), torch.min(mt)
show_image(Image(mt),figsize=(5,5))
# # print(mn)

In [ ]:
mt = learn.model.my_tuple[3][0,...].squeeze_().detach()
mx, mn =torch.max(mt), torch.min(mt)
show_image(Image(mt),figsize=(5,5))